In [1]:
import nltk
import pandas as pd
import string 
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.manifold import MDS

In [2]:
import os
parent_directory = os.path.dirname(os.getcwd())
os.chdir(parent_directory)
os.getcwd()

'd:\\When-Hybrid-meets-the-Prompt'

In [3]:
df_ratings = pd.read_csv("Data/ratings.csv")
df_ratings.info()
df_ratings.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4275200 entries, 0 to 4275199
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   item_id    int64  
 1   user_id    float64
 2   user_name  object 
 3   rating     float64
 4   review     object 
dtypes: float64(2), int64(1), object(2)
memory usage: 163.1+ MB


,item_id,user_id,user_name,rating,review
4275195,4795980,121721688.0,Srutirani Khandayat,4.0,NaN
4275196,4795980,121709466.0,Romero Leonados Kulungan,5.0,NaN
4275197,4795980,120927246.0,Akhil A R,5.0,NaN
4275198,4795980,120501416.0,Dommety Dhana Rishitha,5.0,NaN
4275199,4795980,119744130.0,Cami Aynur Ziyatxan,5.0,NaN


In [4]:

df_reviews = df_ratings.dropna().reset_index(drop=True)
df_reviews.info()
df_reviews.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728191 entries, 0 to 728190
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   item_id    728191 non-null  int64  
 1   user_id    728191 non-null  float64
 2   user_name  728191 non-null  object 
 3   rating     728191 non-null  float64
 4   review     728191 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 27.8+ MB


,item_id,user_id,user_name,rating,review
728186,3662908,118520988.0,Kim Dion,5.0,Nice and short
728187,3662908,82774866.0,Sidharth sejwal,5.0,Yeah it’s a good course for Adavance beginners...
728188,3662908,78363934.0,Usman Ali Khan,5.0,i like the way it was done even it was not eve...
728189,3662908,73201302.0,Joydeep Paul,5.0,Getting better understanding and motivation. ...
728190,4795980,123450478.0,Bilal Baig,5.0,it was for the begining he taught very well


In [5]:
# def clean_text(text):
#     # text = re.sub(r"[Ââ€™œ¢\t]","", text)
#     text = re.sub(r'[^\x00-\x7F]+'," ", text)
#     text = re.sub(r'[*+]+',"", text)
#     text = ' '.join(text.split())
#     text = text.lower()
#     return text

# df_ratings["item_description"] = df_ratings["item_description"].apply(clean_text)
# df_ratings.head()

In [6]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=200000,
                                 min_df=0.1,
                                 use_idf=True)

tfidf_matrix = tfidf_vectorizer.fit_transform(df_reviews.review)

In [7]:
tfidf_matrix.toarray()
tfidf_matrix.shape

(728191, 15)

In [8]:
pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index = df_reviews.user_id)

,and,course,for,good,great,in,is,it,of,the,this,to,very,was,you
user_id,,,,,,,,,,,,,,,
152405496.0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
152373866.0,0.000000,0.349332,0.419127,0.376535,0.0,0.0,0.000000,0.000000,0.421983,0.334236,0.000000,0.333392,0.399484,0.000000,0.0
152367574.0,0.389933,0.000000,0.502980,0.000000,0.0,0.0,0.474254,0.458228,0.000000,0.000000,0.000000,0.400093,0.000000,0.000000,0.0
152336042.0,0.000000,0.000000,0.000000,0.520194,0.0,0.0,0.000000,0.527518,0.000000,0.000000,0.000000,0.000000,0.000000,0.671657,0.0
152303730.0,0.563818,0.000000,0.000000,0.000000,0.0,0.0,0.342870,0.331284,0.000000,0.000000,0.000000,0.578508,0.346596,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118520988.0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
82774866.0,0.281852,0.303023,0.727130,0.326619,0.0,0.0,0.000000,0.331217,0.000000,0.000000,0.000000,0.289196,0.000000,0.000000,0.0
78363934.0,0.172256,0.185195,0.222196,0.000000,0.0,0.0,0.209506,0.404852,0.000000,0.177192,0.222728,0.000000,0.000000,0.773212,0.0


In [9]:
num_clusters = 5

km = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100, n_init=1)

km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [10]:
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e'}

cluster_names = {0: 'Cluster 0', 
                 1: 'Cluster 1', 
                 2: 'Cluster 2', 
                 3: 'Cluster 3', 
                 4: 'Cluster 4'}